<a href="https://colab.research.google.com/github/dahyun22/yolov8-pose/blob/main/yolo_pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ultralytics

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import time

model = YOLO('yolov8n-pose.pt')

cap = cv2.VideoCapture(0)

# 키포인트 인덱스
# NOSE, L_EYE, R_EYE, L_EAR, R_EAR, L_SHOULDER, R_SHOULDER, L_ELBOW, R_ELBOW, L_WRIST, R_WRIST, L_HIP, R_HIP, L_KNEE, R_KNEE, L_ANKLE, R_ANKLE = range(17)
NOSE, L_SHOULDER, R_SHOULDER, L_ELBOW, R_ELBOW, L_WRIST, R_WRIST, L_HIP, R_HIP, L_KNEE, R_KNEE, L_ANKLE, R_ANKLE = 0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16

# 거리 계산 함수
def calculate_distance(a, b):
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

# 각도 계산 함수
def calculate_angle(a, b, c):
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# 발차기 탐지
def detect_kick(keypoints, prev_keypoints, fps):
    hip_height = (keypoints[L_HIP][1] + keypoints[R_HIP][1]) / 2
    ankle_height = min(keypoints[L_ANKLE][1], keypoints[R_ANKLE][1])

    # 1. 발목 높이 < 엉덩이 높이 ( y 값, 높이 반비례 )
    if ankle_height > hip_height:
        return False

    # 2. 발목 속도 = 거리 / 시간
    if prev_keypoints is not None:
        time_interval = 1 / 3
        left_ankle_velocity = calculate_distance(keypoints[L_ANKLE], prev_keypoints[L_ANKLE]) / time_interval
        right_ankle_velocity = calculate_distance(keypoints[R_ANKLE], prev_keypoints[R_ANKLE]) / time_interval

        # 더 큰 값 = 차는 발
        max_ankle_velocity = max(left_ankle_velocity, right_ankle_velocity)
        velocity_threshold = 300  # !!!!!!!!조정  1/3 초 동안 300픽셀
        if max_ankle_velocity < velocity_threshold:
            return False

    # 3. 발차기 하는 다리 무릎 각도 확인
    left_knee_angle = calculate_angle(keypoints[L_HIP], keypoints[L_KNEE], keypoints[L_ANKLE])
    right_knee_angle = calculate_angle(keypoints[R_HIP], keypoints[R_KNEE], keypoints[R_ANKLE])

    kicking_knee_angle = min(left_knee_angle, right_knee_angle)
    if kicking_knee_angle > 120:
        return False
    stand_leg_angle = max(left_knee_angle, right_knee_angle)
    if stand_leg_angle < 160:
        return False

    return True

# 연속된 프레임에서 발차기 감지  ( claude 그대로 )
class KickDetector:
    def __init__(self, fps, detection_duration=0.5, detection_threshold=0.7):
        self.fps = fps
        self.detection_duration = detection_duration
        self.detection_threshold = detection_threshold
        self.detection_window = deque(maxlen=int(fps * detection_duration))
        self.prev_keypoints = None

    def detect(self, keypoints):
        is_kicking = detect_kick(keypoints, self.prev_keypoints, self.fps)
        self.detection_window.append(1 if is_kicking else 0)
        self.prev_keypoints = keypoints

        if len(self.detection_window) == self.detection_window.maxlen:
            detection_ratio = sum(self.detection_window) / len(self.detection_window)
            return detection_ratio >= self.detection_threshold
        return False


# 펀치 탐지
def detect_punch(keypoints, prev_keypoints, fps):
    # 팔 곧게 펴져 있는지
    def extended_arm(shoulder, elbow, wrist, threshold=160):
        angle = calculate_angle(shoulder, elbow, wrist)
        return angle > threshold
    # 팔이 같은 x 좌표에 있는지 (없어도 되는지 물어봐야) -> 이방향으로 있는지
    def points_arm(a, b, c, threshold=10):
        return max(abs(a[0] - b[0]), abs(b[0] - c[0]), abs(a[0] - c[0])) < threshold

    time_interval = 1/3  # 1/3초
    frames_per_calculation = int(fps / 3)  # 1/3초 동안의 프레임 수

    # 왼팔, 오른팔 검사
    for side in ['left', 'right']:
        if side == 'left':
            shoulder, elbow, wrist = keypoints[L_SHOULDER], keypoints[L_ELBOW], keypoints[L_WRIST]
            prev_wrist = prev_keypoints[L_WRIST] if prev_keypoints is not None else None
        else:
            shoulder, elbow, wrist = keypoints[R_SHOULDER], keypoints[R_ELBOW], keypoints[R_WRIST]
            prev_wrist = prev_keypoints[R_WRIST] if prev_keypoints is not None else None

        if extended_arm(shoulder, elbow, wrist): # 일자로 펴져 있으면
            if points_arm(shoulder, elbow, wrist): # -> 이방향이면
                if prev_wrist is not None: # 이전 정보 있으면
                    distance = calculate_distance(wrist, prev_wrist)
                    velocity = distance / time_interval  # 속도 = 거리 / 시간
                    velocity_threshold = 600  # 단위: 픽셀/초 이것도 조정~~~~~
                    if velocity > velocity_threshold:
                        return True, side, velocity

    return False, None, 0

# 쓰러짐 탐지
def detect_fall(keypoints, prev_keypoints, fps, fall_history=None):
    # nose = hip = ankle 비슷한 y 좌표 공유하는지
    # 갑자기 좌표 변경
    # 안되면 nose 위치만으로
    def calculate_vertical_ratio(nose, hip, ankle):
        head_hip_distance = abs(nose[1] - hip[1])
        hip_ankle_distance = abs(hip[1] - ankle[1])
        if hip_ankle_distance == 0:
            return float('inf')
        return head_hip_distance / hip_ankle_distance

    hip_y = (keypoints[L_HIP][1] + keypoints[R_HIP][1]) / 2
    ankle_y = min(keypoints[L_ANKLE][1], keypoints[R_ANKLE][1])
    current_ratio = calculate_vertical_ratio(keypoints[NOSE], (0, hip_y), (0, ankle_y))

    if prev_keypoints is None:
        if fall_history is None:
            return False, [current_ratio]
        else:
            fall_history.append(current_ratio)
            return False, fall_history

    prev_hip_y = (prev_keypoints[L_HIP][1] + prev_keypoints[R_HIP][1]) / 2
    prev_ankle_y = min(prev_keypoints[L_ANKLE][1], prev_keypoints[R_ANKLE][1])
    prev_ratio = calculate_vertical_ratio(prev_keypoints[NOSE], (0, prev_hip_y), (0, prev_ankle_y))

    ratio_change = prev_ratio - current_ratio

    if fall_history is None:
        fall_history = [ratio_change]
    else:
        fall_history.append(ratio_change)
        if len(fall_history) > int(fps * 2):  # 2초 동안의 이력만 유지
            fall_history.pop(0)

    fall_threshold = 0.5
    min_frames = int(fps * 0.5)
    if len(fall_history) >= min_frames:
        recent_changes = fall_history[-min_frames:]
        if all(change > fall_threshold for change in recent_changes):
            return True, fall_history

    return False, fall_history


# 점프 탐지
# def detection_jump(keypoints, prev_keypoints, fps):
# 발목 위치, ankle - knee - hip 각도, Nose 위치



# main
prev_keypoints = None
fall_history = None
prev_time = time.time()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # YOLOv8
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes
        if len(boxes) > 0:  # 사람이 감지되면
            keypoints = r.keypoints.data[0].cpu().numpy()

            is_kicking = detect_kick(keypoints, prev_keypoints, fps)
            if is_kicking:
                cv2.putText(frame, "Kick Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            is_punching, punch_side = detect_punch(keypoints, prev_keypoints, fps)
            if is_punching:
                cv2.putText(frame, f"Punch Detected! ({punch_side})", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            is_falling, fall_history = detect_fall(keypoints, prev_keypoints, fps, fall_history)
            if is_falling:
                cv2.putText(frame, "Fall Detected!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            prev_keypoints = keypoints

    # 프레임 표시
    cv2.imshow("Abnormal Behavior Detection Test", frame)
    # 종료 = 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()